# BioBuilder API Demo

In [1]:
import sys

sys.path.insert(0, '..')

import os
import requests

caravagna_model_filepath = '../biosimulator_processes/model_files/Caravagna2010.xml'
print(os.path.exists(caravagna_model_filepath))

tumor_control_biomodel_id = 'BIOMD0000000749'
requests.get('https://www.ebi.ac.uk/biomodels/BIOMD0000000749', headers={'accept': 'application/json'}).json()

True


{'name': 'Reppas2015 - tumor control via alternating immunostimulating and immunosuppressive phases',
 'description': 'The paper describes a model of tumor control via alternating immunostimulating and immunosuppressive phases. \r\nCreated by COPASI 4.25 (Build 207) \r\n\r\nThis model is described in the article: \r\nIn silico tumor control induced via alternating immunostimulating and immunosuppressive phases\r\nAI Reppas, JCL Alfonso, and H Hatzikirou\r\nVirulence 7:2, 174--186\r\n\r\nAbstract: \r\nDespite recent advances in the field of Oncoimmunology, the success potential of immunomodulatory therapies against cancer remains to be elucidated. One of the reasons is the lack of understanding on the complex interplay between tumor growth dynamics and the associated immune system responses. Toward this goal, we consider a mathematical model of vascularized tumor growth and the corresponding effector cell recruitment dynamics. Bifurcation analysis allows for the exploration of model’s d

#### **_Experiment 1_**:  Here we cross a boundary in the stack that is a biological simulation. We go from model configuration, to experiment. Thus, this tooling sits at that level: both experiment specification AND experiment execution. It's not just a way to specify an experiment, but it is also a way to run it, given the many knobs and buttons that you can use predefine and customize the way the actual model is being solved. Our users are seeking to be involved in an experiment as a "stack". We should make a bigger distinction between terms like "model".

In [2]:
from process_bigraph import pp
from biosimulator_processes.biosimulator_builder import BuildPrompter
from biosimulator_processes import SedDataModel as sed 

CobraProcess registered successfully.
CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.


In [4]:
# 1a. define a model for the process composition. In this case, just one model to be re-used as configuration for the processes we create:

simple_tc_model = sed.TimeCourseModel(model_source=sed.BiomodelID(value=tumor_control_biomodel_id))

pp(simple_tc_model)

AttributeError: type object 'SedDataModel' has no attribute 'BiomodelID'

In [ ]:
# 1b. define a TimeCourse process instance using the above object as a parameter. The other parameter is method. See BasiCO documentation for more details on solvers

simple_tc_process = sed.TimeCourseProcess(model=simple_tc_model, method='tauleap')

In [ ]:
# >> The process model instance is viewable as a dataclass...

pp(simple_tc_process)

In [ ]:
# >> ...or a dict:

pp(simple_tc_process.to_dict())

In [ ]:
# 2. instantiate the prompter:

prompter = BuildPrompter()

In [ ]:
# 3. add process(es) to the bigraph with the Time Course model instance we created above. For now, just one process will be added.

prompter.add_single_process(config=simple_tc_process)

In [ ]:
# 4. Inspect the builder instance within prompter:

pp(prompter.builder_instance)

In [ ]:
# 5. Visualize the fully-connected composition:

prompter.visualize_bigraph()

In [ ]:
# 6. Generate a composite engine and use to execute the bigraph that we just created:

prompter.run()

In [ ]:
# 6. Save the composite to a document:

prompter.builder_instance.write(filename='demo_tumor_control_composite_2')

#### **_Experiment 2_**: Load an SBML model from a specified model filepath and add Model changes to the composite before adding it to the bigraph. Here, we expect the user to be familiar enough with the model file they are passing to make individual species/parameter/reaction changes for specific species types. In the Caravagna model, for example, the species involved are T, E, I. Let's change the initial concentration for some of these as an example of model changes:

In [ ]:
from biosimulator_processes.data_model import TimeCourseModelChanges, ModelChanges, ModelChange

In [ ]:
# first make the timecourse model which is easily configured with objects related to model changes
adjusted_tc_model_from_file = TimeCourseModel